# Outline
1. Import Dependencies
2. Prepare Training DataGenerator
2. Build Model
4. Train Model
5. Make Predictions


## Import Dependencies:

In [ ]:
import os
import pandas as pd
# visulization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPool2D ,AveragePooling2D, Flatten, Add
from keras.layers import Dropout, BatchNormalization, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
TRAIN_DIR = '../input/state-farm-distracted-driver-detection/imgs/train/'
TEST_DIR = '../input/state-farm-distracted-driver-detection/imgs/test/'

In [ ]:
IMG_HEIGHT = 240
IMG_WIDTH = 320

## Prepare Training Data Generator:

In [ ]:
EPOCHS=10
BATCH_SIZE=32

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH),
                                color_mode='grayscale', classes=None, class_mode='categorical',
                                batch_size=BATCH_SIZE, shuffle=True, subset='training')

valid_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH),
                                color_mode='grayscale', classes=None, class_mode='categorical',
                                batch_size=BATCH_SIZE, shuffle=True, subset='validation')

## Build Model:

Things to consider  in keras model building:
- No Dropout after Conv layer
- Use dropout after dense layer (use mostly at the end of arch to not loose data)
- Use BatchNorm before any activation function

### Regularizers:
- Dropout
- Weigth Decay (L2) i.e. weights should be smaller. Penalizes model complexity.
- BatchNorm (is a most)
- Data Augmentation: e.g. fix lightning in images


## Basic CNN
conv > batch_norm > relu


Room for improvement:
- Progressive Resizing
- Data Augmentation

In [ ]:
MODELS_DIR = "saved_models"
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

### Setup Callbacks:

In [ ]:
filepath = MODELS_DIR+'/epoch{epoch:02d}-loss{loss:.2f}-val_loss{val_loss:.2f}.hdf5'
# checkpoint
model_checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                                   save_best_only=True, save_weights_only=False, mode='min', period=1)

# early stopping: patience = epochs
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1,
                               mode='min', baseline=None, restore_best_weights=True)

In [ ]:
def plot_model_loss(history):
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()
    

In [ ]:
# use stride 2 in the middle to reduce size and increase no. of filters
# use avgpool at the end (not maxpool)
def seq_conv_block(model, filters=32):
    model.add(Conv2D(filters=filters, kernel_size=(3,3), strides=2, padding="same"))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation("relu"))
    return model

# all conv layers  with strides=1
model = Sequential(name="seq_conv_rmsprop")

model.add(Conv2D(input_shape=(IMG_HEIGHT,IMG_WIDTH,1), filters=16, kernel_size=(3,3), padding="same"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))

model = seq_conv_block(model, filters=32)
model = seq_conv_block(model, filters=64)
model = seq_conv_block(model, filters=128)

model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(500))
model.add(Dropout(0.5))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))

model.add(Dense(10, activation="softmax"))

# sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)
# optimizer = RMSprop(learning_rate=0.001)
# adad = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

## Train model:

In [ ]:
history_v1 = model.fit_generator(train_generator,
                         steps_per_epoch = train_generator.samples // BATCH_SIZE,
                         epochs = EPOCHS, 
                         callbacks=[early_stopping, model_checkpoint],
                         verbose = 1,
                         validation_data = valid_generator,
                         validation_steps = valid_generator.samples // BATCH_SIZE)

In [ ]:
# load saved model
# model = load_model('saved_models/epoch03-loss0.10-val_loss0.07.hdf5')

In [ ]:
plot_model_loss(history_v1.history)

In [ ]:
loss, accuracy = model.evaluate_generator(valid_generator, steps=valid_generator.samples // BATCH_SIZE)
print("Loss:",loss)
print("Accuracy:", accuracy)

## Make Prediction:

In [ ]:
os.makedirs('test/all_files')
!cp -r $TEST_DIR test/all_files/

In [ ]:
TEST_DIR = 'test'
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(TEST_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH),
                                color_mode='grayscale', classes=None, class_mode=None,
                                batch_size=BATCH_SIZE, shuffle=False)




In [ ]:
test_generator.reset()
# test_generator.filenames

In [ ]:
def prepare_submission_df(predictions, ids):
    result_df = pd.DataFrame(predictions, columns=["c0","c1","c2","c3","c4","c5","c6","c7","c8","c9"])
    result_df['img'] = ids
    return result_df

In [ ]:
predictions = model.predict_generator(test_generator, steps=len(test_generator.filenames)/BATCH_SIZE)

In [ ]:
ids = [os.path.basename(p) for p in test_generator.filenames]
final_df = prepare_submission_df(predictions, ids)

In [ ]:
final_df.to_csv("submission.csv", index=False)

In [ ]:
# !pip install kaggle
# !kaggle competitions submit -c state-farm-distracted-driver-detection -f submission.csv -m "First Submission."

## Resnet like model

In [ ]:
def conv_layer(inputs, filters=16, num_strides=1):
    return Conv2D(filters=filters, kernel_size=(3,3), strides=num_strides, padding='same')(inputs)

def conv_block(inputs, filters=16, num_strides=1):
    x = conv_layer(inputs, filters, num_strides)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    return x
    
def resnet_block(inputs, filters=16):
    x_shortcut = inputs
    x = conv_block(inputs, filters)
    x = BatchNormalization(axis=-1)(x)
    x = Add()([x,x_shortcut]) # skip connection
    x = Activation('relu')(x)
    return x
    

inputs = Input(shape=(IMG_HEIGHT,IMG_WIDTH,1))

output_0 = conv_block(inputs=inputs, filters=16)

output_1 = conv_block(output_0, filters=32, num_strides=2)
output_1 = resnet_block(output_1, filters=32)

output_2 = conv_block(output_1, filters=64, num_strides=2)
output_2 = resnet_block(output_2, filters=64)

output_3 = conv_block(output_2, filters=128, num_strides=2)
output_3 = resnet_block(output_3, filters=128)

output_3 = AveragePooling2D(pool_size=(2,2), strides=(2,2))(output_3)

output_4 = Flatten()(output_3)
output_4 = Dropout(0.5)(output_4)

output_5 = Dense(500)(output_4)
output_5 = Dropout(0.5)(output_5)
output_5 = BatchNormalization(axis=-1)(output_5)
output_5 = Activation('relu')(output_5)

output_6 = Dense(10, activation='softmax')(output_5)

res_model = Model(inputs=inputs, outputs=output_6, name="res_model")

res_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
res_model.summary()

In [ ]:
# history_v2 = res_model.fit_generator(train_generator,
#                          steps_per_epoch = train_generator.samples // BATCH_SIZE,
#                          epochs = EPOCHS, 
#                          callbacks=[early_stopping, model_checkpoint],
#                          verbose = 1,
#                          validation_data = valid_generator,
#                          validation_steps = valid_generator.samples // BATCH_SIZE)
# model_v2 = res_model

In [ ]:
# plot_model_loss(history_v2.history)